### A Transition HMM (THMM)



In a transition HMM, the probabilities are associated with the edges
(connections between nodes) instead of the nodes. Each node has an
identifier (0-7) and a state (A,B,C).

![img](./img/hmm5.png)

Example: calculate the probability for a sequence ABAB. This path
through the network is associated with the node sequence 0 > 1 > 3
> 7, or the transition probabilities $0.6 \times 1.0 \times 0.7 = 0.42$.

Put differently, the sequence ABAB has a transition probability of
42%, while each individual node has 100%, and B:7 has 0%. The
probabilities for all possible network paths add up to 1:
![img](./img/hmm6.png)

| Emission|Probability|
|---|---|
| ABAA|0.18|
| ABAB|0.42|
| ACAA|0.048|
| ACAB|0.112|
| ACBB|0.08|
| ACCB|0.16|
|---|---|
| Total|1.|



### Python data structure for THMM



The THMM is defined as an (mutable) ordered set of (immutable) tuples
(x,y) where x is the state of the node, and y is a set of (mutable)
key-value pairs (k,v) where k is the identifier of a next node in the
sequence, and v is the probability associated with that transition (a
dictionary in Python).

Here, 'mutable' ('immutable') means that the structure can (not) be
added or removed once it's been created. Strings e.g. are immutable
(yet ordered), while lists are mutable:



In [1]:
strng = 'Hello'
[print(f'{i}: {item}') for i,item in enumerate(strng)]

0: H
1: e
2: l
3: l
4: o

In [1]:
# Try to change 'H' to 'h'
#strng[0] = 'h'  # 'str object does not support item assignment
new_strng = strng.replace('H','h')
print(new_strng[0])

h

But for lists:



In [1]:
word = 'hello'
char = list(word)
print(char)
print(type(char))
# change first character of the list
char[0] = 'H'
print(char)

['h', 'e', 'l', 'l', 'o']
<class 'list'>
['H', 'e', 'l', 'l', 'o']

Back to the THMM: To encode node 0 in the state 'A' and its potential
transitions to states 'B' or 'C' with probabilities 0.3 and 0.7:



In [1]:
# node 0 in state A with transitions to node 1 or node 2
('A', {1:0.6, 2:0.4})

### Creating a THMM



For the demonstration, we use a simpler THMM:

-   Node 0 is in state A
-   Node 1 is in state B
-   Node 2 is in state C
-   Node 3 is in state D

All transition probabilities except 0 > 1 and 0 > 2 are 1.0.
![img](./img/hmm7.png)

The function `THMM` creates this network using the tuple + dictionary structure:



In [1]:
def THMM():
    '''Create transition hidden markov model for four nodes 0-3 with
    four states A,B,C,D and transition probabilities 0 > 1 of 0.3 and
    0 > 2 of 0.7.
    '''
    # initialize empty dictionary
    net = {}

    # define set of nodes as a dictionary with: tuple of state letter
    # and dictionary of key = node, value = transition probability
    net['begin'] = ('',{0:1.0}) # stateless - transition to node 0 only
    net[0] = ('A',{1:0.3,2:0.7}) # transition to nodes 1 or 2
    net[1] = ('B',{3:1.0}) # transition to nodes 1 or 2
    net[2] = ('C',{3:1.0}) # transition to nodes 1 or 2
    net[3] = ('D',{'end':1.0}) # transition to nodes 1 or 2
    net['end'] = ('',{}) # stateless - no transition from the end node
    return net

To create the net, run the function on the HMM object `hmm`:



In [1]:
net = THMM()
print(type(net))
[print(values) for values in net.values()]

<class 'dict'>
('', {0: 1.0})
('A', {1: 0.3, 2: 0.7})
('B', {3: 1.0})
('C', {3: 1.0})
('D', {'end': 1.0})
('', {})

You can draw this easily using a simple drawing library like [pyvis](https://pyvis.readthedocs.io/en/latest/) or
[plotly](https://plotly.com/python/network-graphs/).

Checking:



In [1]:
print(net.keys()) # keys = identify node data
print(net['begin']) # dictionary data for key = 'begin'
print(net[0][1].keys()) # keys of inside dictionary: identifier +
                        # transition information
print(net[0][1][1]) # probability to go from node 0 to 1

dict_keys(['begin', 0, 1, 2, 3, 'end'])
('', {0: 1.0})
dict_keys([1, 2])
0.3

### Computing probability for a THMM



To compute the probability, the network needs to know which node
follows the current node. This is implemented in `next_node`:



In [1]:
def next_node(net,k,ask):
    '''Return probability `hit` to transition to node k

    net: Transition Hidden Markov Model
    k: identifier of node to transition to
    ask: asked state to transition to
    '''
    # extract nodes to which transition is possible from k
    t = net[k][1].keys()

    # initialize empty list
    hit = []

    # iterate over the nodes to which transition is possible
    for i in t:
        # check if a node is in the asked state `ask` (A,B,C,D)
        if net[i][0] == ask:
            # if match, extract node identifier and probability
            hit = i, net[k][1][i]
            break
    return hit

Use the function e.g. to find the probability to go to the node with
the next letter in the sequence ('B'):



In [1]:
print(next_node(net,0,'B'))

(1, 0.3)

To consider the entire sequence, use the function `thmm_prob`: it
receives the THMM and the test string `in_string`. The search begins
with the begin node, considers each letter in the sequence, retrieves
the transition probabilities and multiplies them.



In [1]:
def thmm_prob(net, in_string):
    L = len(in_string)
    pbs = 1.0
    k = 'begin'
    for i in range(L):
        tran = next_node(net, k, in_string[i])
        k = tran[0] 
        pbs *= tran[1]
    return pbs

Example function call:



In [1]:
print(thmm_prob(net, 'ABD'))
print(thmm_prob(net, 'ACD'))

0.3
0.7